The Monty Hall Problem analyzed using a Bayesian Network
---
adapted from https://github.com/jmschrei/pomegranate

In [1]:
from pomegranate import BayesianNetwork, DiscreteDistribution, ConditionalProbabilityTable, State

In [8]:
# helper function to pretty print observation results
def update(network, observations, variables=None):
    beliefs = network.forward_backward(observations)
    for state, dist in zip(network.states, beliefs):
        if variables is None or state.name in variables:
            fixed = {}
            for k, v in dist.parameters[0].items():
                fixed[k] = "{:.2}".format(v)
            print("{:<15}\t{}".format(state.name, fixed))

Setup the discrete probability distributions based on the Monty Hall Problem model

In [3]:
prize = DiscreteDistribution({'1': 1/3, '2': 1/3, '3': 1/3})
contestant = DiscreteDistribution({'1': 1/3, '2': 1/3, '3': 1/3})
host = ConditionalProbabilityTable([
        ['1', '1', '1', 0.0], ['2', '1', '1', 0.0], ['3', '1', '1', 0.0],
        ['1', '1', '2', 0.5], ['2', '1', '2', 0.0], ['3', '1', '2', 1.0],
        ['1', '1', '3', 0.5], ['2', '1', '3', 1.0], ['3', '1', '3', 0.0],
        ['1', '2', '1', 0.0], ['2', '2', '1', 0.5], ['3', '2', '1', 1.0],
        ['1', '2', '2', 0.0], ['2', '2', '2', 0.0], ['3', '2', '2', 0.0],
        ['1', '2', '3', 1.0], ['2', '2', '3', 0.5], ['3', '2', '3', 0.0],
        ['1', '3', '1', 0.0], ['2', '3', '1', 1.0], ['3', '3', '1', 0.5],
        ['1', '3', '2', 1.0], ['2', '3', '2', 0.0], ['3', '3', '2', 0.5],
        ['1', '3', '3', 0.0], ['2', '3', '3', 0.0], ['3', '3', '3', 0.0]
    ], [contestant, prize]) 

Create state objects (network nodes)

In [4]:
prize_state = State(prize, name='prize')
contestant_state = State(contestant, name='contestant')
host_state = State(host, name='host')

Construct the actual network by adding the nodes then creating the directed edges between them

In [5]:
monty = BayesianNetwork("Monty Hall Problem")
monty.add_states([prize_state, contestant_state, host_state])
monty.add_transition(prize_state, host_state)
monty.add_transition(contestant_state, host_state)
monty.bake()

The contestant chooses door number 1. The contestant's CPT is now frozen and the host's has updated to reflect this information. As the contestant and prize distributions are independent, this has no effect on the prize's CPT.

In [9]:
update(monty, {"contestant": "1"})

prize          	{'1': '0.33', '2': '0.33', '3': '0.33'}
contestant     	{'1': '1.0', '2': '0.0', '3': '0.0'}
host           	{'1': '0.0', '2': '0.5', '3': '0.5'}


The host then reveals door number 2. The host's CPT is now frozen as well, though this time the prize's CPT has changed to reflect the influence of both the host's decision **AND** the contestant's, as observing the host allowed the contestant's choice to flow to the prize's CPT.

In this case, the contestant should always switch.

In [10]:
update(monty, {"contestant": "1", "host": "2"})

prize          	{'1': '0.33', '2': '0.0', '3': '0.67'}
contestant     	{'1': '1.0', '2': '0.0', '3': '0.0'}
host           	{'1': '0.0', '2': '1.0', '3': '0.0'}


If we were to tune in late and **ONLY** see the host's choice but not which door the contestant had chosen, we are left with a different belief as to what the contestant should do, namely that it doesn't matter if they switch or stay.

In [11]:
update(monty, {"host": "2"})

prize          	{'1': '0.5', '2': '0.0', '3': '0.5'}
contestant     	{'1': '0.5', '2': '0.0', '3': '0.5'}
host           	{'1': '0.0', '2': '1.0', '3': '0.0'}
